<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [176]:
import pandas as pd
import psycopg2
import urllib.parse as req
import pandas as pd 
import requests
import warnings
warnings.filterwarnings('ignore')

In [178]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [179]:
# текст запроса
query_3_1 = f'''select 
           count(id) "количество вакансий"
           from vacancies 
        '''

In [180]:
df = pd.read_sql_query(query_3_1, connection)
df 

,количество вакансий
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [181]:
query_3_2 = f'''select 
           count(distinct id) "количество работодателей"
           from employers
        '''

In [182]:
df = pd.read_sql_query(query_3_2, connection)
df

,количество работодателей
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [183]:
query_3_3 = f'''select 
           count(distinct id) "количество регионов"
           from areas 
        '''

In [184]:
df = pd.read_sql_query(query_3_3, connection)
df

,количество регионов
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [185]:
query_3_4 = f'''select 
           count(distinct id) "сферы деятельности"
           from industries  
        '''

In [186]:
df = pd.read_sql_query(query_3_4, connection)
df

,сферы деятельности
0,294


- **В базе содержится 49197 вакансий, 23501 работодателей, из 1362 Регионов, и охватывают 294 сферы деятельности. Более детальный анализ показывает, что в основном вакансии из сферы IT**
- **В разрезе предварительного анализа было бы интересно посмотреть как долго вакансия находится в базе, как быстро работодатели находят специалистов различных уровней и сколько всего вакансий содержится в базе на текущий период, чтобы понимать какой вес занимают вакансии в области IT**

***

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [187]:
query_4_1 = f'''select 
           a.name "регион",
           count(*) "кол-во вакансий"
           from vacancies v join areas a on v.area_id = a.id 
           group by a.id
           order by 2 desc           
        '''

In [188]:
df = pd.read_sql_query(query_4_1, connection)
df.head(5)

,регион,кол-во вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [189]:
query_4_2 = f'''select 
           count(id) "вакансии с з/п"
           from vacancies
           where salary_from is not null or salary_to is not null 
        '''

In [190]:
df = pd.read_sql_query(query_4_2, connection)
df

,вакансии с з/п
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [191]:
query_4_3 = f'''select
          'нижняя граница з/п' "показатель", 
           round(AVG(salary_from)) "значение"
           from vacancies
           union
           select
          'верхняя граница з/п' , 
           round(AVG(salary_to))
           from vacancies
        '''

In [192]:
df = pd.read_sql_query(query_4_3, connection)
df

,показатель,значение
0,верхняя граница з/п,110537.0
1,нижняя граница з/п,71065.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [193]:
query_4_4 = f'''select 
           schedule "рабочий график",
           employment "тип трудоустройства",
           count(*) "кол-во вакансий"
           from vacancies
           group by schedule, employment 
           order by 3 desc   
        '''

In [194]:
df = pd.read_sql_query(query_4_4, connection)
df.head(2)

,рабочий график,тип трудоустройства,кол-во вакансий
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [195]:
query_4_5 = f'''select 
           experience "опыт работы",
           count(*) "кол-во вакансий"
           from vacancies
           group by experience 
           order by 2     
        '''

In [196]:
df = pd.read_sql_query(query_4_5, connection)
df

,опыт работы,кол-во вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


- **В пятерку лидеров-городов по колличеству вакансий входят Москва, Санкт-Питербург, Минск, Новосибирск, Алмааты Это самые крупные региональные центры СНГ**

- **24073 вакансии имеют верхнюю или нижнюю границу зарплаты. это означает что в половине вакансий не указанна зарплата. Возможно, это связано с тем что работодатели готовы рассматривать специалистов с разным уровнем подготовки, обучать новичков или же нанимать специалистов высокого уровня т.к таких специалистов на рынке мало. Поэтому окончательную з/п не указывают**

- **Средняя нижняя граница равна 71065 руб, а средняя верхняя 110537 руб. Это соответствует и сегодняшнему положению на рынке. На сегодняшний день средняя по отрасли около 80000 рублей по региону Москва**

- **Второй по популярности график работы и тип трудоустройства является удаленная работа - Полная занятость. Это логично, во первых политическая, экономическая и эпидемиологическая ситуация, во вторых IT специалисту необходимо постоянно развиваться, а работая удаленно у него больше времени на себя и доп.проекты. Но в то же время это только 14% от всех вакансий. Поэтому можно сделать вывод, что большая часть предпочитает работать в офисе**

- **По колличеству вакансий от требуемого опыта вакансии распределены следующим образом Более 6 лет — Нет опыта — От 3 до 6 лет — От 1 года до 3 лет что показывает что большинство вакансий имеют требование опыта работы От 1 года до 3 лет. Скорее всего это связано с тем, что работодатели хотят видеть специалиста у которого уже есть опыт работы в команде, но в то же время он еще обучаем.**

***

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [197]:
query_5_1 = f'''select 
           e.name "название компании",
           count(v.id) "кол-во вакансий"
           from employers e join vacancies v on v.employer_id = e.id	
           group by e.name 
           order by 2 desc   
        '''

In [198]:
df = pd.read_sql_query(query_5_1, connection)
df.head(5)

,название компании,кол-во вакансий
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [199]:
query_5_2 = f'''select 
             a.name "название региона",
             count(v.id) "кол-во вакансий",
             count(distinct e.id) "кол-во работадателей"
   	         from areas a
 	         left join employers e on a.id = e.area
 	         left join vacancies v on v.area_id = a.id
             group by a.id
             having count(v.id) = 0 and count(distinct e.id) != 0
             order by 3 desc
            '''

In [200]:
df = pd.read_sql_query(query_5_2, connection)
df.head(5)

,название региона,кол-во вакансий,кол-во работадателей
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Беларусь,0,18


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [201]:
query_5_3 = f'''select 
                e.name "название компании",
                count(distinct v.area_id) "кол-во регионов"
                from employers e
                join vacancies v on v.employer_id = e.id
                group by 1
                order by 2 desc
         '''

In [202]:
df = pd.read_sql_query(query_5_3, connection)
df.head()

,название компании,кол-во регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [203]:
query_5_4 = f'''select
                count(distinct e.id) "не указана сфера деятельности"
                from employers e
                left join employers_industries ei on ei.employer_id = e.id
                where industry_id is null 
        '''

In [204]:
df = pd.read_sql_query(query_5_4, connection)
df.head()

,не указана сфера деятельности
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [205]:
query_5_5 = f'''select
                e.name "название компании",
                count(ei.industry_id) "кол-во сфер деятельности"
                from employers e
                join employers_industries ei on ei.employer_id = e.id
                group by e.name
                having count(ei.industry_id) > 3
                order by 2, 1     
        '''

In [206]:
df = pd.read_sql_query(query_5_5, connection)
df.head()

,название компании,кол-во сфер деятельности
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [207]:
query_5_6 = f'''select
                count(distinct ei.employer_id) "разрабртка прогр. обеспечения"
                from employers_industries ei
                join industries i on ei.industry_id = i.id
                group by i.id	
                having i.name = 'Разработка программного обеспечения'
        '''

In [208]:
df = pd.read_sql_query(query_5_6, connection)
df.head()

,разрабртка прогр. обеспечения
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [209]:
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров')
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
url
response = requests.get(url)
city = pd.read_html(url)[1]['Город']
lst_city = []
for i in city:
    lst_city.append(i)
tuple(lst_city)

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Челябинск',
 'Красноярск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Омск',
 'Краснодар',
 'Воронеж[a]',
 'Пермь',
 'Волгоград')

In [210]:
query_5_7 = f'''(select 
           a.name "регион",
           count(*) "кол-во вакансий"
           from vacancies v 
           join areas a on v.area_id = a.id
           join employers e on v.employer_id = e.id
           where e.name = 'Яндекс' and a.name in ('Москва',
           'Санкт-Петербург',
           'Новосибирск',
           'Екатеринбург',
           'Казань',
           'Нижний Новгород',
           'Челябинск',
           'Красноярск',
           'Самара',
           'Уфа',
           'Ростов-на-Дону',
           'Омск',
           'Краснодар',
           'Воронеж',
           'Пермь',
           'Волгоград')
           group by a.id)
           
           union all
           
           (select
           'Total',
            count(v.id) 
           from vacancies v 
           join areas a on v.area_id = a.id
           join employers e on v.employer_id = e.id
           where e.name = 'Яндекс' and a.name in ('Москва',
           'Санкт-Петербург',
           'Новосибирск',
           'Екатеринбург',
           'Казань',
           'Нижний Новгород',
           'Челябинск',
           'Красноярск',
           'Самара',
           'Уфа',
           'Ростов-на-Дону',
           'Омск',
           'Краснодар',
           'Воронеж',
           'Пермь',
           'Волгоград'))
           order by 2
           '''  

In [211]:
df = pd.read_sql_query(query_5_7, connection)
df.head(20)

,регион,кол-во вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


- **Среди компаний, находящихся в исследуемой базе данных, наибольшее количество вакансий размещают IT компании, банки и компании занимающиеся добычей полезных ископаемых**

- **Часть крупных компаний стремятся разместить вакансии во многих регионах. Это связано как с предоставлением услуг в этих регионах, так и вероятно с желанием охватить более широкий круг людей, что увеличивает вероятность получения лучших специалистов в штат**

- **Треть работадателей не указали сферу деятельности. Возможно в списке сфер деятельности не присутствует необходимых сфер или компания является широко известной в регионе размещения или компания осуществляет деятельность по разным направлениям**

***

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [212]:
query_6_1 = f'''select
                count(id) "Data вакансии"
                from vacancies
                where name ilike '%data%' or name ilike '%данн%'
        '''

In [213]:
df = pd.read_sql_query(query_6_1, connection)
df

,Data вакансии
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [214]:
query_6_2 = f'''select
                count(id) "Data для Junior"
                from vacancies
                where 
                (name ilike '%data%scientist%' or 
                name ilike '%data%science%' or 
                name ilike '%исследователь%данных%' or 
                (name like '%ML%' and name not ilike '%HTML%') or 
                name ilike '%machine%learning%' or 
                name ilike '%машинн%обучен%')
                and (name ilike '%junior%' or experience = 'Нет опыта' or employment = 'Стажировка' )      
        '''

In [215]:
df = pd.read_sql_query(query_6_2, connection)
df

,Data для Junior
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [216]:
query_6_3 = f'''select
                count(id) "вакансии с SQL и PG"
                from vacancies
                where 
                (name ilike '%data%scientist%' or 
                name ilike '%data%science%' or 
                name ilike '%исследователь%данных%' or 
                (name like '%ML%' and name not ilike '%HTML%') or 
                name ilike '%machine%learning%' or 
                name ilike '%машинн%обучен%')
                and (key_skills ilike '%SQL%' or key_skills ilike '%postgres%')      
        '''

In [217]:
df = pd.read_sql_query(query_6_3, connection)
df

,вакансии с SQL и PG
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [218]:
query_6_4 = f'''select
                count(id) "вакансии Python"
                from vacancies
                where 
                (name ilike '%data%scientist%' or 
                name ilike '%data%science%' or 
                name ilike '%исследователь%данных%' or 
                (name like '%ML%' and name not ilike '%HTML%') or 
                name ilike '%machine%learning%' or 
                name ilike '%машинн%обучен%')
                and (key_skills ilike '%python%')       
        '''

In [219]:
df = pd.read_sql_query(query_6_4, connection)
df

,вакансии Python
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [220]:
query_6_5 = f'''select
       round(avg(length(key_skills)-length(replace(key_skills, chr(9), ''))+1), 2) "кол-во навыков"
       from vacancies
       where 
       (name ilike '%data%scientist%' or 
        name ilike '%data%science%' or 
        name ilike '%исследователь%данных%' or 
        (name like '%ML%' and name not ilike '%HTML%') or 
        name ilike '%machine%learning%' or 
        name ilike '%машинн%обучен%')
        '''

In [221]:
df = pd.read_sql_query(query_6_5, connection)
df

,кол-во навыков
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [222]:
query_6_6 = f'''select
       experience "опыт работы",
       round(avg(coalesce((salary_from + salary_to)/2, salary_from, salary_to))) "средняя з/п"
       from vacancies 
       where 
       (salary_from is not null or salary_to is not null) and
       (name ilike '%data%scientist%' or 
        name ilike '%data%science%' or 
        name ilike '%исследователь%данных%' or 
        (name like '%ML%' and name not ilike '%HTML%') or 
        name ilike '%machine%learning%' or 
        name ilike '%машинн%обучен%')
        group by experience
        '''

In [223]:
df = pd.read_sql_query(query_6_6, connection)
df

,опыт работы,средняя з/п
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


- **3.5% связаны с профессией дата-сайентиста. Так как количество используемой информации в мире увеличивается, количество вакансий в этой сфере будет расти**
- **Несмотря на 1771 вакансию, которые относится к DS, лишь 51 вакансия может быть предложена соискателям без опыта работы. Это 3% всех вакансий в этой сфере. Таким образом, направления деятельности связанные с обучением профессии дата-сайентиста являются выгодными для создания. Так же эта цифра может сказать о том, что рост зарплаты с увелечением опыта работы будет существенным, что было показано в одном из пунктов выше**
- **Среди ключевых навыков Python указывают лишь в 20% случаев, а требования к работе с языком SQL или СУБД Postgres в 11%. Анализа прочих требований и ключевых навыков не проводилось, но можно предугадать что в требованиях, указывают более специфичные вещи. Тогда как эти навыки являются основными и их владение подразумевеются без упоминания**
- **Величина зарплаты дата-сайентиста с опытом работы более трёх лет близится к 250000 рублей, что более чем в два раза превосходит среднее верхнее значение зарплатной вилки из всех профессий безотносительно опыта**

***

# Общий вывод по проекту

- **10 ноября 2022 года веб-сервис для хостинга IT-проектов и их совместной разработки GitHub опубликовал отчёт Octoverse, в котором в числе прочего приводится рейтинг самых востребованных в 2022 году языков программирования. Топ - 6 в следующем порядке: самым популярным является JavaScript. Далее идут Python, Java, Typescript и C#. На шестой позиции располагается C++  Посмотрим, как эти языки расположатся в рейтинге наших требуемых навыкох в вакансиях**
- **Так же добавили знание Git и SQL**

In [224]:
query_vacancies1 = f'''
         select
         'Java' "навык", 
         count(id) "кол-во вакансий"
         from vacancies
         where key_skills ilike '%Java%' and key_skills not ilike '%JavaScript%'
         union
         select
         'Git', 
         count(id) 
         from vacancies
         where key_skills ilike '%Git%'
         union
         select
         'Python', 
         count(id) 
         from vacancies
         where key_skills ilike '%Python%'
         union
         select
         'SQL', 
         count(id) 
         from vacancies
         where key_skills ilike '%SQL%'
         union
         select
         'C++', 
         count(id) 
         from vacancies
         where key_skills ilike '%C++%'
         union
         select
         'JavaScript', 
         count(id) 
         from vacancies
         where key_skills ilike '%JavaScript%'
         union
         select
         'TypeScript', 
         count(id) 
         from vacancies
         where key_skills ilike '%TypeScript%'
         union
         select
         'C#', 
         count(id) 
         from vacancies
         where key_skills ilike '%C#%' 
         
         order by 2
        '''

In [225]:
df = pd.read_sql_query(query_vacancies1, connection)
df

,навык,кол-во вакансий
0,TypeScript,1024
1,C++,1162
2,C#,1521
3,Java,2171
4,Python,3461
5,JavaScript,3648
6,Git,5027
7,SQL,10882


- **Мы видим, что такая закономерность сохраняется и в нашей выборке. Так же мы видим, что для 25% вакансий необходимо знание SQL - это большая цифра, т.е скорее всего для всех разработчиков, аналитиков и всех, кто непосредственно связан с созданиями запросов и написания кодов, а вот работа с Git нужна меньше, но возможно, часть работодателей ее просто не указала**


- **В данном проекте были составлены многочисленных SQL-запросы позволившие изучить рынок вакансий. Проект хорошо показывает, возможности по извлечению нужной информации хранящейся в базе данных, применяя предложение SELECT языка запросов SQL**

- **В целом, если говорить про рынок вакансий, то видно что наиболее высокая зарплата у дата-сейнтистов но войти в данную отрасль без опыта достаточно сложно поскольку для начинающих специалистов очень мало предложений, а для специалистов с опытом наоборот**

- **Так же мы видим что предложения в основном сконцентрированы в самых больших городах, что естественно. Всегда специалисты высокого уровня и знаний стремились выбраться из регионов. Кроме того в больших городах создается больше условий для развития IT проектов**

